In [1]:
from dataset import TextDataset
import matplotlib.pyplot as plt
import numpy as np
from dataset import EEDataset
import pandas as pd
import torch
import torch.nn as nn

In [2]:
df = pd.read_csv("dataset/split_train.csv")

In [3]:
vc = dict(df['PRODUCT_TYPE_ID'].value_counts())
id_to_ind = {}
default_ind = 0
for k, v in vc.items():
    if v > 10:
        id_to_ind[k] = default_ind
        default_ind += 1
    else:
        id_to_ind[k] = default_ind

In [4]:
train_set = EEDataset(path="dataset/split_train.csv", id_to_ind=id_to_ind, default_ind=default_ind, drop_big=True)
val_set = EEDataset(path="dataset/split_val.csv", id_to_ind=id_to_ind, default_ind=default_ind)

========> LOADING DATASET <========
========> LOADING DATASET <========


In [6]:
import torch
import torch.nn as nn
from tqdm import tqdm
from model import EntityEmbedding
model = EntityEmbedding(embedding_dim=32, num_embeddings=len(id_to_ind))
model.load_state_dict(torch.load('checkpoints/ee_no_transform/model_best_epoch_5.pth.tar')['state_dict'])

c:\Users\Pranav\miniconda3\envs\amazonml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

In [7]:
import pandas as pd
test = pd.read_csv('dataset/test.csv')

In [9]:
test_preds = pd.DataFrame({'PRODUCT_ID': test['PRODUCT_ID'], 'PRODUCT_LENGTH': 0})
for i in tqdm(range(0, len(test), 16)):
    x = test["PRODUCT_TYPE_ID"][i:i+16]
    inds = [0]*len(x)
    for j, k in enumerate(x):
        if k in id_to_ind.keys():
            inds[j] = id_to_ind[k]
        else:
            inds[j] = default_ind
    x = torch.tensor(inds)
    out = model(x).squeeze().detach().numpy()
    out = (out * val_set.std) + val_set.mean
    test_preds['PRODUCT_LENGTH'][i:i+16] = out

100%|██████████| 45921/45921 [00:23<00:00, 1914.04it/s]


In [11]:
test_preds.to_csv('test_preds_type_id.csv', index=False)

In [112]:
test_preds = pd.DataFrame({'PRODUCT_ID': test['PRODUCT_ID'], 'PRODUCT_LENGTH': 39.7})
test_preds.to_csv('dataset/test_preds.csv', index=False)

In [24]:
def transformed_mape(y_true, y_pred):
    #y_true = (np.log(y_true) - val_set.mean) / val_set.std
    return np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + 1e-8))

In [89]:
model.eval()
model(torch.tensor([1, 34, 455, 299, 0, 12501]))

tensor([[0.0011],
        [0.0013],
        [0.0016],
        [0.0011],
        [0.0012],
        [0.0015]], grad_fn=<AddmmBackward0>)

In [92]:
mape = 0
total = 0
for i in range(0, len(val_set), 8):
    x = val_set.data["PRODUCT_TYPE_ID"][i:i+8]
    inds = [0]*len(x)
    for j, k in enumerate(x):
        if k in id_to_ind.keys():
            inds[j] = id_to_ind[k]
        else:
            inds[j] = default_ind
    x = torch.tensor(inds)
    out = model(x).squeeze().detach().numpy()
    out = (out * val_set.std) + val_set.mean
    y = val_set.data["PRODUCT_LENGTH"][i:i+8]
    total += len(y)
    #print(x)
    sum = np.sum(np.abs(y - out) / (np.abs(y) + 1e-8))
    mape += sum
 
print(out[:10], y[:10])
print(mape / total)

KeyboardInterrupt: 

In [30]:
transformed_mape(val_set.data['PRODUCT_LENGTH'][:1], 40)

0.946526315722277

In [46]:
df['PRODUCT_LENGTH'].mean(), df['PRODUCT_LENGTH'].std(), df['PRODUCT_LENGTH'].max(), df['PRODUCT_LENGTH'].min()

(4164.737850855155, 1465722.4180191627, 1885801400.0, 1.0)